In [ ]:
!pip install ultralytics datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 629.1/629.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.7 MB/s eta 0:00:00


In [ ]:
import os
from PIL import Image
from ultralytics import YOLO
from datasets import load_dataset

"""
datasets/
  -images
      -train
          -0.png
          -1.png
          ..
          ..
      -validation
          -n1.png
          -n2.png
  -labels
      -train
      -validation

0.txt
1 x_n y_n w_n h_n
2 x_n y_n w_n h_n
..
"""
def dump_images_and_labels(data, split):
  data =data[split]
  for i,example in enumerate(data):
    image = example['image']
    labels = example['litter']['label']
    bboxes = example['litter']['bbox']
    targets = []
    for label,box in zip(labels,bboxes):
      targets.append(f"{label} {box[0]} {box[1]} {box[2]} {box[3]}")
      with open(f"datasets/labels/{split}/{i}.txt",'w') as f:
        for target in targets:
          f.write(target+"\n")

      image.save(f"datasets/images/{split}/{i}.png")

if __name__ == "__main__":
  dataset = load_dataset("Kili/plastic_in_river")
  os.makedirs("datasets/images/train",exist_ok=True)
  os.makedirs("datasets/images/validation",exist_ok=True)
  os.makedirs("datasets/labels/train",exist_ok=True)
  os.makedirs("datasets/labels/validation",exist_ok=True)

  dump_images_and_labels(dataset,"train")
  dump_images_and_labels(dataset,"validation")

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
model = YOLO('yolov8m.pt')

## Train the model
model.train(
    data='/content/plastic.yaml',
    epochs=20,
    imgsz=(1280,720),
    batch=4,
    optimizer="Adam",
    lr0 = 1e-3
)

Ultralytics YOLOv8.0.134 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/plastic.yaml, epochs=20, patience=50, batch=4, imgsz=(1280, 720), save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dyna